In [139]:
import xml.etree.ElementTree as ET
import os, os.path


## you are currently overriding entries on the same protein in different organisms! This is occuring around 10 times

In [140]:
file_directory = os.listdir(r'C:\Users\odhra\Documents\graph_project\allosteric_reasonings\data\raw\ASD\ASD_Release_062015_XF')

In [141]:
file_path = r'C:\Users\odhra\Documents\graph_project\allosteric_reasonings\data\raw\ASD\ASD_Release_062015_XF'

In [142]:
paths = []
for filename in file_directory:
    paths.append(os.path.join(file_path,filename))


In [143]:
entries = {}
missing = []
Protein_ID_list = []
for ASD_file in paths:

    try:
        root = ET.parse(ASD_file).getroot()
        Protein_ID = root.find('UniProt_ID').text
        Organism = root.find('Organism').text
        Gene = root.find('Gene')[0].text
        Gene_ID = root.find('Gene')[2].text
        
        modulator_list = []

        try:
            for modulators in root.find('Modulator_List'):
                modulator_list.append((modulators[0].text, modulators[1].text))    
        except:
            pass

        entries[Protein_ID] = {
            'Organism' : Organism,
            'Gene' : Gene,
            'Gene_ID' : Gene_ID,
            'Modulators' : modulator_list
        }

        Protein_ID_list.append((Protein_ID))
        
        
    except:
        missing.append(ASD_file)

print(len(missing))


43


In [159]:
Protein_ID_list = list(dict.fromkeys(Protein_ID_list))

In [160]:
print(len(Protein_ID_list))

1426


In [162]:
Protein_ID_list[0]

'P0AB91'

In [169]:
entries['P0AB91']['Modulators']

[('ASD00020003', 'Inhibitor'),
 ('ASD00320001', 'Inhibitor'),
 ('ASD00690002', 'Inhibitor'),
 ('ASD02170001', 'Inhibitor')]

In [203]:
edge_list_with_IDs = []
for protein in Protein_ID_list:
    interactions = entries[protein]['Modulators']
    for ASD in interactions:
        edge_list_with_IDs.append([protein, ASD[0], ASD[1]])


In [204]:
nodes= list(set([item[0] for item in edge_list_with_IDs] + [item[1] for item in edge_list_with_IDs]))

In [205]:
node_hash_map = {}
for node_index, node in enumerate(nodes):
    
    node_hash_map[node] = node_index

In [206]:
node_hash_map['ASD01811158']

1

In [207]:
edge_list = []
for protein in Protein_ID_list:
    interactions = entries[protein]['Modulators']
    for ASD in interactions:
        edge_list.append([
            node_hash_map[protein],
            node_hash_map[ASD[0]],
             ASD[1]])


In [156]:
print(len(entries))

1426


In [144]:
    modulator_dict = {}
    for modulators in root.find('Modulator_List'):
        modulator_dict[modulators[0].text] = modulators[1].text
    print(modulator_dict)

{'ASD05550002': 'Regulator', 'ASD06980001': 'Regulator'}


In [145]:
    modulator_list = []
    for modulators in root.find('Modulator_List'):
        modulator_list.append((modulators[0].text, modulators[1].text))
    print(modulator_list)

[('ASD05550002', 'Regulator'), ('ASD06980001', 'Regulator')]


In [146]:
dictionary = 

SyntaxError: invalid syntax (<ipython-input-146-4d131ce30d24>, line 1)

In [147]:
root.find('UniProt_ID').text

'G7TWB8'

In [148]:
root.find('Alias_List')[0].text

TypeError: 'NoneType' object is not subscriptable

In [149]:
for modulator_compounds in root.find('Modulator_List'):
    print(modulator_compounds)
    

<Element 'Modulator' at 0x0000013A5E432540>
<Element 'Modulator' at 0x0000013A5E432310>


In [150]:
for modulator_compounds in root.find('Modulator_List'):
    print(modulator_compounds[0].text)
    print(modulator_compounds[1].text)

ASD05550002
Regulator
ASD06980001
Regulator


In [151]:
print(len(root.find('Modulator_List')))

2


In [152]:
for child in root:
    print(child)

<Element 'Organism_ID' at 0x0000013A5E5F9A90>
<Element 'Organism' at 0x0000013A5E5F9130>
<Element 'Organism_Latin' at 0x0000013A5E5F9680>
<Element 'UniProt_ID' at 0x0000013A5E5F9630>
<Element 'Rfam_ID' at 0x0000013A5E5F9180>
<Element 'Allosteric_Activator_Count' at 0x0000013A5FFD2C20>
<Element 'Allosteric_Inhibitor_Count' at 0x0000013A5FFD2F90>
<Element 'Allosteric_Regulator_Count' at 0x0000013A5FFD2590>
<Element 'Molecule_Seq' at 0x0000013A5FFD2D10>
<Element 'Gene' at 0x0000013A5FFD2B30>
<Element 'PDB_List' at 0x0000013A5FFD2680>
<Element 'AllostericSite_List' at 0x0000013A5E432DB0>
<Element 'Enzyme_Nomenclature_List' at 0x0000013A5E4324F0>
<Element 'Modulator_List' at 0x0000013A5E432BD0>
<Element 'Create_Date' at 0x0000013A5E432900>
